In [ ]:
import requests
import datetime
import pandas as pd
import matplotlib.pylab as plt

sign = '31ea8aa540a61e824f4d61ca0e23aa6602b599dd53309e27ce2b7d3936cc92f81c9e152dbbf325c3c961fc748c1b019c95b9900b59673a35ff54950760244e30769418079afe1458bab107d09897782e9e661166880043b4e1dce872af48110ad2fadb67fbb831375122a1b06214026958ed837b2f4e8b2a882f14aed03047b368e3ec2c2590c5eb13f57eea11714f9a'
tenantID = '2'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8', '_tenantId': tenantID, '_sign' : sign, 'accept':'*/*'}
pageSize = '50000'

url_api = 'http://ft-api.prod.trimblevi.com:20551/openapi/v2/'
url_fleets = url_api + 'fleets?'
fleet_names = ['Shaw%20Regional%20-%201333','Shaw%20IPT%20-%201331', 'Shaw%20OTR%20-%202108']
#fleet_names = ['Boise%20Cascade%20-%204241']
#fleet_names = ['Quickway%20Dist%20-%201508']
fleetIds=[]
if(len(fleet_names)>0):
    url_fleets = url_fleets + '&fleetName='
    first = True
    for fleet_name in fleet_names :
        url_fleet = url_fleets + fleet_name
        response = requests.get(url_fleet, headers=headers)

        response_json = response.json()
        fleetIds.append(response_json['data'][0]['fleetId'])

start_date = datetime.datetime(year=2024, month=5, day=1)
start_date_str = start_date.strftime("%Y-%m-%d")

end_date =  datetime.datetime(year=2024, month=5, day=7)
end_date_str = end_date.strftime("%Y-%m-%d")

url_alarms = url_api + 'alarms?alarmTypes=0'

if(len(fleetIds)>0):
    url_alarms = url_alarms + '&includeSubFleet=true&fleetIds='
    first = True
    for fleet_id in fleetIds :
        if(first):
            url_alarms = url_alarms + fleet_id
            first=False
        else:
            url_alarms = url_alarms + '%2C'+ fleet_id

url_alarms = url_alarms + '&endTime='+end_date_str+'T23%3A59%3A59Z&startTime='+start_date_str+'T00%3A00%3A00Z' + '&pageSize=1000'

page = 1
full_list_df = pd.DataFrame()
while ((page == 1) or (response.text.find('alarmAddInfo') > 0)):
    url_final = url_alarms + '&page=' + str(page)
    response = requests.get(url_final, headers=headers)
    page_df = pd.json_normalize(response.json(), ["data"])
    full_list_df = pd.concat([full_list_df, page_df], ignore_index=True, sort=False)
    page += 1

full_list_df['gps.speed'] = full_list_df['gps.speed'] * 0.621371
full_list_df = full_list_df[['vehicleNumber', 'alarmAddInfo.channelInfo.channels', 'gps.speed', 'startTime', 'endTime']]
full_list_df = full_list_df.rename(columns={'vehicleNumber': 'License plate number', 'alarmAddInfo.channelInfo.channels': 'Channel', 'gps.speed': 'Speed', 'startTime': 'Start time', 'endTime': 'End time'})
display(full_list_df)

In [ ]:
# Boxplot data
def get_box_plot_data(labels, bp):
    rows_list = []

    for i in range(len(labels)):
        dict1 = {}
        dict1['label'] = labels[i]
        dict1['lower_whisker'] = bp['whiskers'][i*2].get_ydata()[1]
        dict1['lower_quartile'] = bp['boxes'][i].get_ydata()[1]
        dict1['median'] = bp['medians'][i].get_ydata()[1]
        dict1['upper_quartile'] = bp['boxes'][i].get_ydata()[2]
        dict1['upper_whisker'] = bp['whiskers'][(i*2)+1].get_ydata()[1]
        rows_list.append(dict1)

    return pd.DataFrame(rows_list)

camera_failure_freq = full_list_df.groupby(["License plate number"])["License plate number"].value_counts()
camera_failure_freq = camera_failure_freq.to_frame().reset_index()
camera_failure_freq = camera_failure_freq.sort_values(by='count', ascending=False)
plt.yscale('log')
bp = plt.boxplot(camera_failure_freq['count'])
bp_info = get_box_plot_data(['count'],bp)
display(bp_info)

In [ ]:
# Count alarms from outlier devices

outlier_count_df = camera_failure_freq[camera_failure_freq['count']>bp_info['upper_whisker'][0]]
outlier_count_df = outlier_count_df.sort_values(by='count', ascending=False)
display(outlier_count_df)

In [ ]:
# Create outliers and no outliers list

outlier_list_df = full_list_df[full_list_df["License plate number"].isin(outlier_count_df["License plate number"])]
no_outlier_list_df = full_list_df[~ full_list_df["License plate number"].isin(outlier_count_df["License plate number"])]

In [ ]:
# Alarms number according to the speed by channel 

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(6, 8), dpi=100)
fig.tight_layout(pad=3.5)

selected_df = full_list_df

channel_1_df = selected_df[selected_df['Channel'] == '1']
channel_2_df = selected_df[selected_df['Channel'] == '2']
channel_4_df = selected_df[selected_df['Channel'] == '4']
channel_1_df.plot.hist(ax=axes[0], column=["Speed"], ylabel='Alarms', xlabel='Speed (mph)',legend=False, rwidth=0.9)
channel_2_df.plot.hist(ax=axes[1], column=["Speed"], ylabel='Alarms', xlabel='Speed (mph)',legend=False, rwidth=0.9)
channel_4_df.plot.hist(ax=axes[2], column=["Speed"], ylabel='Alarms', xlabel='Speed (mph)',legend=False, rwidth=0.9)

In [ ]:
# Select list to create the charts

selected_df = outlier_list_df

In [ ]:
# Count alarms by channel

channels_df = selected_df.groupby(["Channel"])["Channel"].value_counts()
channels_df = channels_df.to_frame().reset_index()
display(channels_df)

In [ ]:
# Count alarms by channel per vehicle

channelVehicles_df = selected_df[['Channel', 'License plate number']]
channelVehicles_df = channelVehicles_df.drop_duplicates(subset=['License plate number'])
channelVehicles_df = channelVehicles_df.groupby(['Channel']).agg('count').reset_index()
display(channelVehicles_df)

In [ ]:
# Measure alarms duration

duration_df = selected_df[['License plate number', 'Speed', 'Start time', 'End time']].copy()
duration_df[['Start time', 'End time']] = duration_df[['Start time', 'End time']].apply(pd.to_datetime)
duration_df['Duration'] = duration_df['End time'] - duration_df['Start time']
no_na_duration_df = duration_df.dropna()
no_na_duration_df = no_na_duration_df.sort_values(by='Duration', ascending=True)
display(no_na_duration_df)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8, 7), dpi=80)
fig.tight_layout(pad=3.5)
#plt.suptitle('Shaw Fleet (outliers removed)')
selected_df.plot.hist(ax=axes[0,0], column=["Speed"], ylabel='Alarms', xlabel='Speed (mph)',legend=False, rwidth=0.9)
no_na_duration_df['Duration'].dt.seconds.plot.hist(ax=axes[0,1], rwidth=0.9, grid=False, xlabel='Duration (s)', ylabel='Alarms')
channels_df.plot.bar(ax=axes[1,0],  x='Channel', y='count', ylabel='Alarms', rot=0, legend=False)
channelVehicles_df.plot.bar(ax=axes[1,1], x='Channel', y='License plate number', ylabel='Vehicles', rot=0, legend=False)

In [ ]:
# No endtime tretment

values = []
values.append(full_list_df['End time'].isna().sum())
values.append(full_list_df.shape[0])

fig, ax = plt.subplots()
pie=ax.pie(values, labels=["End time unavailable", "End time available"], autopct='%1.1f%%', pctdistance = 0.75, counterclock=False, startangle=-270, wedgeprops={'width':0.5, 'edgecolor':'white', 'linewidth': 2})
fig.tight_layout()

no_endtime_df = full_list_df.drop(full_list_df[~full_list_df['End time'].isna()].index)
no_endtime_df = no_endtime_df.groupby(["License plate number"])["License plate number"].value_counts()
no_endtime_df = no_endtime_df.to_frame().reset_index()
no_endtime_df = no_endtime_df.sort_values(by='count', ascending=False)
display(no_endtime_df)